# 7.利用郵遞區號進行分析使用KMeans演算法將客戶分群印出分群結果

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from sklearn.cluster import KMeans ,DBSCAN
from sklearn import cluster,metrics
from mlxtend.frequent_patterns import apriori ,association_rules

sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
plt.rcParams['axes.unicode_minus'] = False # 正常顯示負號

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

# 改style要在改font之前
# plt.style.use('seaborn')

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

# EDA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_FR7.csv')

df

# Processing Data

In [ ]:
# 決定針對顧客的郵遞區號、性別、年齡、平均長途話費來進行分析

x = df[['郵遞區號' ,'性別' ,'年齡' ,'平均長途話費']]

# 評估指標

# 輪廓係數(Silhouette Evaluation)

輪廓係數的值越接近1，表示聚類的效果越好

In [ ]:
fig = plt.figure(figsize=(10,8))
silhouette_avg = []
for i in range(2,9):
    kmeans_fit = KMeans(n_clusters=i, init ='k-means++', max_iter=300,  n_init=10,random_state=0).fit(x)
    silhouette_avg.append(metrics.silhouette_score(x, kmeans_fit.labels_))
print(f"輪廓係數 = {silhouette_avg}")

plt.plot(range(2, 9), silhouette_avg, 'bx-')
plt.title('silhouette')
plt.xlabel('No of clusters')
plt.ylabel('Avg')
plt.show()

# WCSS 損失函數 (within-cluster sum of squares)

Elbow method，手肘法
透過嘗試多種類別個數，並將相應的 WCSS 記錄下來並且畫出，理想上最佳的K值應該是最大轉折處

In [ ]:
wcss = []
for i in range(1, 9):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42).fit(x)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 9), wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('No of clusters')
plt.ylabel('WCSS')
# 在每個點上標示相應的群數
for k, w in zip(range(1, 9), wcss):
    plt.text(k, w, f'({k},{int(w)})', ha='center', va='bottom')
plt.show()

結合上面的輪廓係數評估，我們認為聚類數量為4是一個合適的選擇

# KMeans Clustering

In [ ]:
k = 4
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(x)

df['Cluster'] = clusters

df.head()

# Plot

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['郵遞區號'], df['Cluster'], c=df['Cluster'], cmap='viridis', s=50)
plt.xlabel('郵遞區號')
plt.ylabel('Cluster')
plt.title('Scatter Plot of 郵遞區號 and Cluster')
plt.show()

圖表得知各群的郵遞區號分布

In [ ]:
# 根據 Cluster 分組，計算每個群組中男女的數量
grouped = df.groupby('Cluster')['性別'].value_counts().unstack()

grouped.plot(kind='bar', figsize=(10, 6), colormap='viridis')
plt.xlabel('Cluster')
plt.ylabel('Count')
plt.title('各群組中男女性別的分佈')
plt.xticks(rotation=0)
plt.legend(title='性別')
plt.show()

第一群和第二群女性多於男性，三四群則是男性多於女性

In [ ]:
# 根據 Cluster 分組，計算每個群組的年齡
grouped = df.groupby('Cluster')['年齡'].mean()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='年齡', data=df, palette='viridis')
plt.xlabel('Cluster')
plt.ylabel('年齡')
plt.title('各群組的年齡箱型圖')
plt.show()

一二三群的年齡箱型圖長的差不多，第四群的Q3(上四分位數)以及Q1(下四分位數)的差距較其他三群大，意味著年齡分散程度較大

In [ ]:
# 根據 Cluster 分組，計算每個群組的平均長途話費

grouped = df.groupby('Cluster')['平均長途話費'].mean()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='平均長途話費', data=df, palette='viridis')
plt.xlabel('Cluster')
plt.ylabel('平均長途話費')
plt.title('各群組的平均長途話費箱型圖')
plt.show()

各群的平均長途話費箱型圖都長的差不多，但第四群的中位數略高其他群